## look at rates and thermo

In [48]:
import cantera as ct
import numpy as np 

In [49]:
!source activate rmg_env && ck2cti --input="../base/chemkin/chem_annotated-gas.inp" --surface="../base/chemkin/chem_annotated-surface.inp" --output="../base/cantera/chem.cti"

Wrote CTI mechanism file to '../base/cantera/chem.cti'.
Mechanism contains 157 species and 2474 reactions.
Validating mechanism...PASSED.


In [45]:
rxn = "HCCO(150) + OH(24) <=> CH2(82) + CO2(12)".replace("+", "").replace("<=>","").split()

In [46]:
rxn

['HCCO(150)', 'OH(24)', 'CH2(82)', 'CO2(12)']

## check problem reactions 
cantera check prints them in the wrong order for some reason, makes searching hard

In [47]:
reac_inline = 0
comment_lines = []

with open("../base/chemkin/chem_annotated-gas.inp") as f:
    for linenum,line in enumerate(f):
        for s in rxn: 
            if s in line:
                reac_inline+=1
                
        if len(rxn) == reac_inline:
            print(linenum, line)
            
            # if line is not commented, flag so we know to comment it out
            if "!" not in line:
                comment_lines.append(linenum)
                
        reac_inline = 0
        


5995 OH(24)+HCCO(150)=CO2(12)+CH2(82)                    1.000e+00 0.000     0.000    

6002 OH(24)+HCCO(150)=CO2(12)+CH2(82)                    1.000e+00 0.000     0.000    

6009 OH(24)+HCCO(150)=CO2(12)+CH2(82)                    1.000e+00 0.000     0.000    



### check mechanism: 
#### check rate constants

In [55]:
cti_file = '../base/cantera/chem.cti'
T = 700  #400 500, 600, 700, 800 
P = ct.one_atm
X = {'HAN': 0.45947550796503267, # hydroxyl ammonium nitrate
    'NH4NO3': 0.029214215643766223, # ammonium nitrate
    'CH3OH(5)': 0.20628782954534425, # methanol
    'H2O(6)': 0.3050224468458569, # water
    }


gas = ct.Solution(cti_file, "gas")
surf = ct.Interface(cti_file, "surface1", [gas])


gas.TPX = T, P, X
surf.TP = T, P

## look at lists of rates, moles, etc. 

In [56]:
def dict_n_sort(names, values):
    dictionary = {}
    
    for i,j in enumerate(names):
        dictionary[j]=values[i]
        
    dictionary = {k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1], reverse=True)}
    
    return dictionary

### rate constants - gas (forward and reverse)

In [60]:
gas_forw_rate_constants = dict_n_sort(gas.reaction_equations(),gas.forward_rate_constants)
gas_forw_rate_constants

{'CH2OH(86) + NO2(9) <=> S(1115)': 1.1043839871399066e+17,
 'C2H3N(240) <=> C2H3N(242)': 1691569863436.998,
 'OCHCO(199) <=> CO(14) + HCO(22)': 752111513023.4004,
 'H2NO3(461) + HCO(22) <=> CH2O(17) + HNO3(4)': 362000000000.0,
 'C2H5(21) + NO3(132) <=> C2H4(25) + HNO3(4)': 216900000000.0,
 'C2H4N(233) + H(20) <=> C2H5N(229)': 206334390468.31424,
 'C2H4N(243) + H(20) <=> C2H5N(247)': 206334390468.31424,
 'CH2(S)(94) + H(20) <=> CH2(82) + H(20)': 200000000000.0,
 'HNO3(292) <=> HNO3(4)': 185206363844.3897,
 'HCO(22) + HNO3(292) <=> CH2O(17) + NO3(132)': 181000000000.0,
 'CH3O(84) + HCO(22) <=> 2 CH2O(17)': 171711700000.0,
 'CH3OO(170) + OH(24) <=> CH3OH(5) + O2(8)': 170000000000.0,
 'CH3CO(185) + O(52) <=> CH3(18) + CO2(12)': 160000000000.0,
 'CH2(S)(94) + CH2CO(171) <=> C2H4(25) + CO(14)': 160000000000.0,
 'C2H4N(235) + O(52) <=> CH3(18) + HNCO(121)': 160000000000.0,
 'C2H6N(226) + H(20) <=> C2H7N(224)': 154031465068.6809,
 'H(20) + HCCO(150) <=> CH2(S)(94) + CO(14)': 150000000000.0,
 '

In [61]:
gas_rev_rate_constants = dict_n_sort(gas.reaction_equations(),gas.reverse_rate_constants)
gas_rev_rate_constants

{'H2(13) + NCN(92) <=> H(20) + HNCN(93)': 1.1062727017245028e+18,
 'CH(81) + NH3(2) <=> 2 H(20) + H2CN(99)': 5.058186736191016e+17,
 'CH3(18) + M <=> CH2(82) + H(20) + M': 630979118216823.8,
 'CH3NO(230) + M <=> H(20) + H2NCO(234) + M': 447202060400876.7,
 'CH3(18) + M <=> CH(81) + H2(13) + M': 183915149390490.75,
 'H(20) + NO3(132) <=> HNO3(292)': 16343239232089.045,
 'HNCNH(205) + O(52) <=> HNCN(93) + OH(24)': 4746932374120.53,
 'C2H2(173) + M <=> C2H(174) + H(20) + M': 4307647404644.269,
 'HCN(58) + N2(7) <=> CN(60) + H(20) + N2(7)': 4157277591668.53,
 'CH3NO(230) + M <=> HCO(22) + NH2(56) + M': 2965982972849.6226,
 'H(20) + HNCNH(205) <=> H2(13) + HNCN(93)': 813895777546.3223,
 'C2H5N(229) + H(20) <=> C2H5N(228) + H(20)': 406677367762.6309,
 'CH3(18) + H2NO(141) <=> CH3O(84) + NH2(56)': 262993040303.37704,
 '2 H2O(6) <=> H(20) + H2O(6) + OH(24)': 254262008308.83752,
 'C2H4(25) + NO2(9) <=> S(217)': 235845799544.8735,
 'NH3(2) + M <=> H(20) + NH2(56) + M': 225467336751.1237,
 'H(20)

### forward rate constants - surf

In [62]:
surf_forw_rate_constants = dict_n_sort(surf.reaction_equations(),surf.forward_rate_constants)
surf_forw_rate_constants 

{'CH2X(43) + HX(38) <=> CH3X(39) + X(1)': 3.09e+21,
 'CHX(44) + HX(38) <=> CH2X(43) + X(1)': 3.09e+21,
 'CHX(44) + X(1) <=> CX(42) + HX(38)': 3.09e+21,
 'CXHO(45) + X(1) <=> HX(38) + OCX(41)': 3.71e+20,
 'CH2O_X(62) + CXHO(45) <=> CH3O_X(85) + OCX(41)': 3.398e+20,
 'CX(42) + HO_X(30) <=> HX(38) + OCX(41)': 2.8836968933466096e+20,
 'SX(1398) + X(1) <=> HX(38) + SX(1410)': 4.969107382948695e+19,
 'HAN_X(29) + X(1) <=> H3NOX(33) + HNO3_X(34)': 2.747646488001445e+19,
 'HX(38) + O_Pt(36) <=> HO_X(30) + X(1)': 1.8682268601059308e+19,
 'CH3O_X(85) + X(1) <=> CH2O_X(62) + HX(38)': 1.7694032015177228e+19,
 'SX(26) + X(1) <=> H2O(6) + H2O_X(28) + N2O_X(27)': 1.5896714460490359e+19,
 'HAN_X(29) + X(1) <=> H2NOX(31) + HONO(32) + HO_X(30)': 1.5116554891267731e+19,
 'CH2X(43) + HO_X(30) <=> CH3X(39) + O_Pt(36)': 1.029518017169146e+19,
 'HONO(385) + X(1) <=> HNOX(383) + O_Pt(36)': 8.954514726540129e+18,
 'CH4X(555) + HO_X(30) <=> CH3X(39) + H2O_X(28)': 7.929821715952035e+18,
 'CH3O_X(85) + CH4X(555) 

In [63]:
surf_rev_rate_constants = dict_n_sort(surf.reaction_equations(),surf.reverse_rate_constants)
surf_rev_rate_constants

{'N2O_X(27) + X(1) <=> N2_X(51) + O_Pt(36)': 6.51076938624902e+36,
 'SX(26) + X(1) <=> O_Pt(36) + SX(344)': 3.0864898195890556e+36,
 'CH2O_X(62) + SX(314) <=> CH3O_X(85) + SX(1651)': 3.9201565605232943e+34,
 'CH2O_X(62) + SX(314) <=> CH3OX(87) + SX(1651)': 3.6367076347885124e+33,
 'SX(314) + X(1) <=> HX(38) + SX(1651)': 1.4090979152453097e+33,
 'NHO_X(68) + SX(314) <=> H2NOX(290) + SX(1651)': 4.212002560177069e+32,
 'HNO3_X(34) + SX(314) <=> SX(1651) + SX(336)': 1.5832976212933404e+32,
 'NH4NO3 + 2 X(1) <=> O_Pt(36) + SX(344)': 9.83071005048371e+31,
 'NHO_X(68) + SX(314) <=> H2NOX(31) + SX(1651)': 8.268470714630977e+30,
 'HAN_X(29) + X(1) <=> O_Pt(36) + SX(407)': 1.7089298127155612e+28,
 'CH2O_X(62) + SX(1139) <=> CH3O_X(85) + SX(1148)': 1.7065651829739042e+27,
 'NHO_X(68) + SX(1139) <=> H2NOX(290) + SX(1148)': 4.538344749261708e+25,
 'CH2O_X(62) + H2O_X(28) <=> CH3O_X(85) + HO_X(30)': 3.090281897681683e+25,
 'HAN + 2 X(1) <=> O_Pt(36) + SX(407)': 2.9375824995030067e+25,
 'CHX(44) + X(

### equilibrium constants - gas

In [64]:
gas_equilib = dict_n_sort(gas.reaction_equations(),gas.equilibrium_constants)
gas_equilib

{'C2(193) + O2(8) <=> 2 CO(14)': 1.338966414362203e+78,
 'C2O(100) + O2(8) <=> CO(14) + CO2(12)': 2.5192054257616532e+64,
 'C2O(100) + O(52) <=> 2 CO(14)': 2.060876134973527e+63,
 'CN(60) + NO2(9) <=> CO2(12) + N2(7)': 1.783586941028157e+63,
 'CH2(82) + O(52) <=> CO(14) + H2(13)': 3.812687399394879e+54,
 'N(55) + NCO(111) <=> CO(14) + N2(7)': 2.2225181051121808e+53,
 'CH(81) + O(52) <=> CO(14) + H(20)': 2.181057451440338e+53,
 'C2O(100) + NO2(9) <=> CO2(12) + NCO(111)': 1.1951356851359925e+49,
 'CH(81) + HCCO(150) <=> C2H2(173) + CO(14)': 1.955051303564019e+47,
 'C(63) + OH(24) <=> CO(14) + H(20)': 1.3748770121692262e+47,
 'HCCO(150) + N(55) <=> CO(14) + HCN(58)': 1.2971613391429818e+47,
 'C2H(174) + NO(10) <=> CO(14) + HCN(58)': 3.6338820490005735e+46,
 'CH2(S)(94) + N2O(11) <=> CH2O(17) + N2(7)': 2.0829662802919763e+46,
 'C2H(174) + O2(8) <=> 2 CO(14) + H(20)': 3.012705904422083e+45,
 'NCO(111) + NO(10) <=> CO2(12) + N2(7)': 1.6038271700141238e+44,
 'C2H3N(239) + O(52) <=> CH2O(17) +

### equilibrium constants - surface

In [65]:
surf_equilib = dict_n_sort(surf.reaction_equations(),surf.equilibrium_constants)
surf_equilib

{'CXHO(45) + SX(1651) <=> OCX(41) + SX(314)': 1.3979087703714109e+47,
 'NHO_X(68) + SX(1651) <=> NO_Pt(37) + SX(314)': 1.6775409319856134e+46,
 'CH2X(43) + SX(1651) <=> CHX(44) + SX(314)': 8.687391531624235e+45,
 'CH2O_X(62) + SX(1651) <=> CXHO(45) + SX(314)': 5.553239698211795e+45,
 'HAN_X(29) + SX(1651) <=> 2 SX(314)': 2.0064655188412217e+45,
 'CH4OX(83) + SX(1651) <=> CH3OX(87) + SX(314)': 8.607795494921406e+44,
 'H2NOX(290) + SX(1651) <=> HNOX(383) + SX(314)': 3.1212554050027815e+44,
 'HO_X(30) + SX(1651) <=> O_Pt(36) + SX(314)': 4.1696221439560075e+43,
 'NH2_X(48) + SX(1651) <=> NH_X(49) + SX(314)': 4.221646358495245e+42,
 'SX(1398) + SX(1651) <=> SX(1410) + SX(314)': 2.541357063917729e+42,
 'SX(1651) + SX(26) <=> SX(300) + SX(314)': 1.8145361786069576e+42,
 'HONO(385) + SX(1651) <=> NO2_X(50) + SX(314)': 1.0300311994033916e+42,
 'NH_X(49) + SX(1651) <=> N_Pt(35) + SX(314)': 2.0233479962033233e+41,
 'H3NOX(33) + SX(1651) <=> H2NOX(31) + SX(314)': 1.370658836487467e+41,
 'H2_X(77) 

In [67]:
rxn = "CH2OH(86) + NO2(9) <=> S(1115)".replace("+", "").replace("<=>","").split()

In [68]:
reac_inline = 0
comment_lines = []

with open("../base/chemkin/chem_annotated-gas.inp") as f:
    for linenum,line in enumerate(f):
        for s in rxn: 
            if s in line:
                reac_inline+=1
                
        if len(rxn) == reac_inline:
            print(linenum, line)
            
            # if line is not commented, flag so we know to comment it out
            if "!" not in line:
                comment_lines.append(linenum)
                
        reac_inline = 0

1935 ! Flux pairs: CH2OH(86), S(1115); NO2(9), S(1115); 

1937 NO2(9)+CH2OH(86)=S(1115)                            6.895180e+79 -21.017   0.000    



In [ ]:
gas_rop_dict = dict_n_sort(gas.reaction_equations(),gas.reverse_rates_of_progress)
gas_rop_dict

In [ ]:
gas_rop_dict = dict_n_sort(gas.reaction_equations(),gas.forward_rates_of_progress)
gas_rop_dict

In [ ]:
gas_rop_dict = dict_n_sort(surf.reaction_equations(),surf.forward_rates_of_progress)
gas_rop_dict

In [ ]:
surf_rop_dict = dict_n_sort(surf.reaction_equations(),surf.reverse_rates_of_progress)
surf_rop_dict